# Funnel Decomposition Summary Demo

This notebook provides a concise summary of all key decomposition charts:

1. **Lender Booking Decomposition** - Single lender aggregate waterfall (YoY and MoM)
2. **Multi-Lender Comparison** - Aggregate waterfall with lender breakdown
3. **Penetration Decomposition** - Self-adjusted approach (4 individual charts)

All charts are saved as individual PNG files for easy copying and sharing.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import calculators
from lmdi_decomposition_calculator import (
    calculate_decomposition,
    calculate_multi_lender_decomposition
)
from lmdi_penetration_calculator import (
    calculate_penetration_decomposition,
    calculate_multi_lender_penetration_decomposition,
    print_penetration_decomposition
)

# Import visualization functions
from visualization_summary import create_waterfall_grid, _create_aggregate_waterfall
from visualization_lender import create_lender_aggregate_waterfall, create_lender_waterfall_grid
from visualization_penetration import (
    create_penetration_waterfall_grid,
    create_penetration_chart_net_lender_plus_competitor,
    create_penetration_chart_net_all_inclusive,
    create_penetration_chart_net_lender_vs_competitor,
    create_penetration_chart_net_volume_inclusive
)

# Create output directory
output_dir = Path.cwd().parent / 'outputs' / 'summary_charts'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Output directory: {output_dir}")
print("Modules loaded successfully!")

---
## User Configuration

**Specify the lender and analysis date below.** The notebook will automatically calculate:
- **YoY (Year-over-Year)**: Comparing the specified date to the same month last year
- **MoM (Month-over-Month)**: Comparing the specified date to the previous month

In [ ]:
# ============================================================
# USER INPUT: Specify your lender and analysis date here
# ============================================================

LENDER = 'ACA'           # Options: 'ACA', 'ALY', 'CAP'
ANALYSIS_DATE = '2024-06-01'  # Format: 'YYYY-MM-01' (first of month)

# ============================================================

In [ ]:
# Load data
data_path = Path.cwd().parent / 'data' / 'funnel_data_2d_mock_with_nonfinanced.csv'
df = pd.read_csv(data_path)
df['month_begin_date'] = pd.to_datetime(df['month_begin_date'])

# Parse analysis date and calculate comparison dates
analysis_date = pd.to_datetime(ANALYSIS_DATE)
yoy_date = analysis_date - relativedelta(years=1)  # Same month last year
mom_date = analysis_date - relativedelta(months=1)  # Previous month

# Validate inputs
available_lenders = df['lender'].unique().tolist()
available_dates = df['month_begin_date'].unique()

print(f"Data loaded: {len(df)} rows")
print(f"Date range: {df['month_begin_date'].min().date()} to {df['month_begin_date'].max().date()}")
print(f"Available lenders: {available_lenders}")
print()
print("=" * 60)
print(f"Selected Lender: {LENDER}")
print(f"Analysis Date:   {analysis_date.date()}")
print(f"YoY Comparison:  {yoy_date.date()} -> {analysis_date.date()}")
print(f"MoM Comparison:  {mom_date.date()} -> {analysis_date.date()}")
print("=" * 60)

# Validate
if LENDER not in available_lenders:
    raise ValueError(f"Lender '{LENDER}' not found. Available: {available_lenders}")
if analysis_date not in available_dates:
    raise ValueError(f"Date '{analysis_date.date()}' not found in data.")
if yoy_date not in available_dates:
    print(f"WARNING: YoY date {yoy_date.date()} not in data - YoY analysis will be skipped")
if mom_date not in available_dates:
    print(f"WARNING: MoM date {mom_date.date()} not in data - MoM analysis will be skipped")

---
## 1. Single Lender Booking Decomposition (YoY)

LMDI decomposition of booking changes for a single lender - Year over Year comparison.

In [ ]:
# Calculate YoY decomposition
if yoy_date in available_dates:
    yoy_results = calculate_decomposition(
        df=df,
        date_a=str(yoy_date.date()),
        date_b=str(analysis_date.date()),
        lender=LENDER
    )
    
    # Display summary
    meta = yoy_results.metadata
    print(f"\n{LENDER} YoY Booking Decomposition: {yoy_date.date()} -> {analysis_date.date()}")
    print("=" * 60)
    print(f"Period 1 Bookings: {meta['period_1_total_bookings']:,.0f}")
    print(f"Period 2 Bookings: {meta['period_2_total_bookings']:,.0f}")
    print(f"Change:            {meta['delta_total_bookings']:+,.0f}")
    print("\nEffect Breakdown:")
    for _, row in yoy_results.summary.iterrows():
        name = row['effect_type'].replace('_', ' ').title()
        print(f"  {name:<25} {row['booking_impact']:>+10,.0f}")
else:
    print(f"Skipping YoY analysis - date {yoy_date.date()} not available")

In [ ]:
# Create and save YoY waterfall
if yoy_date in available_dates:
    fig, ax = plt.subplots(figsize=(14, 8))
    
    _create_aggregate_waterfall(
        ax=ax,
        summary=yoy_results.summary,
        period_1_bks=meta['period_1_total_bookings'],
        period_2_bks=meta['period_2_total_bookings'],
        title=f'{LENDER} YoY Booking Decomposition: {yoy_date.date()} -> {analysis_date.date()}'
    )
    
    plt.tight_layout()
    
    # Save PNG
    output_file = output_dir / f'{LENDER.lower()}_yoy_booking_waterfall.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    plt.show()

---
## 2. Single Lender Booking Decomposition (MoM)

LMDI decomposition of booking changes for a single lender - Month over Month comparison.

In [ ]:
# Calculate MoM decomposition
if mom_date in available_dates:
    mom_results = calculate_decomposition(
        df=df,
        date_a=str(mom_date.date()),
        date_b=str(analysis_date.date()),
        lender=LENDER
    )
    
    # Display summary
    meta_mom = mom_results.metadata
    print(f"\n{LENDER} MoM Booking Decomposition: {mom_date.date()} -> {analysis_date.date()}")
    print("=" * 60)
    print(f"Period 1 Bookings: {meta_mom['period_1_total_bookings']:,.0f}")
    print(f"Period 2 Bookings: {meta_mom['period_2_total_bookings']:,.0f}")
    print(f"Change:            {meta_mom['delta_total_bookings']:+,.0f}")
    print("\nEffect Breakdown:")
    for _, row in mom_results.summary.iterrows():
        name = row['effect_type'].replace('_', ' ').title()
        print(f"  {name:<25} {row['booking_impact']:>+10,.0f}")
else:
    print(f"Skipping MoM analysis - date {mom_date.date()} not available")

In [ ]:
# Create and save MoM waterfall
if mom_date in available_dates:
    fig, ax = plt.subplots(figsize=(14, 8))
    
    _create_aggregate_waterfall(
        ax=ax,
        summary=mom_results.summary,
        period_1_bks=meta_mom['period_1_total_bookings'],
        period_2_bks=meta_mom['period_2_total_bookings'],
        title=f'{LENDER} MoM Booking Decomposition: {mom_date.date()} -> {analysis_date.date()}'
    )
    
    plt.tight_layout()
    
    # Save PNG
    output_file = output_dir / f'{LENDER.lower()}_mom_booking_waterfall.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    plt.show()

---
## 3. Multi-Lender Booking Comparison (YoY)

Two views:
- **Aggregate waterfall**: Total market with lender breakdown
- **Simple aggregate**: Total market without breakdown

In [ ]:
# Calculate multi-lender YoY decomposition
if yoy_date in available_dates:
    multi_yoy_results = calculate_multi_lender_decomposition(
        df=df,
        date_a=str(yoy_date.date()),
        date_b=str(analysis_date.date())
    )
    
    # Display summary
    print(f"\nMulti-Lender YoY Booking Decomposition: {yoy_date.date()} -> {analysis_date.date()}")
    print("=" * 60)
    print(f"Period 1 Total Bookings: {multi_yoy_results.metadata['aggregate_period_1_bookings']:,.0f}")
    print(f"Period 2 Total Bookings: {multi_yoy_results.metadata['aggregate_period_2_bookings']:,.0f}")
    print(f"Change:                  {multi_yoy_results.metadata['aggregate_delta_bookings']:+,.0f}")
    print(f"\nLenders: {multi_yoy_results.metadata['lenders']}")

In [ ]:
# Create and save simple multi-lender YoY aggregate waterfall
if yoy_date in available_dates:
    fig, ax = plt.subplots(figsize=(14, 8))
    
    _create_aggregate_waterfall(
        ax=ax,
        summary=multi_yoy_results.aggregate_summary,
        period_1_bks=multi_yoy_results.metadata['aggregate_period_1_bookings'],
        period_2_bks=multi_yoy_results.metadata['aggregate_period_2_bookings'],
        title=f'Multi-Lender YoY Aggregate: {yoy_date.date()} -> {analysis_date.date()}'
    )
    
    plt.tight_layout()
    
    # Save PNG
    output_file = output_dir / 'multi_lender_yoy_aggregate_simple.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    plt.show()

In [ ]:
# Create and save multi-lender YoY breakdown waterfall (with stacked lender bars)
if yoy_date in available_dates:
    fig = create_lender_aggregate_waterfall(
        lender_summaries=multi_yoy_results.lender_summaries,
        aggregate_summary=multi_yoy_results.aggregate_summary,
        date_a=str(yoy_date.date()),
        date_b=str(analysis_date.date()),
        period_1_bks=multi_yoy_results.metadata['aggregate_period_1_bookings'],
        period_2_bks=multi_yoy_results.metadata['aggregate_period_2_bookings']
    )
    
    # Save PNG
    output_file = output_dir / 'multi_lender_yoy_breakdown_waterfall.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    plt.show()

---
## 4. Multi-Lender Booking Comparison (MoM)

In [ ]:
# Calculate multi-lender MoM decomposition
if mom_date in available_dates:
    multi_mom_results = calculate_multi_lender_decomposition(
        df=df,
        date_a=str(mom_date.date()),
        date_b=str(analysis_date.date())
    )
    
    # Display summary
    print(f"\nMulti-Lender MoM Booking Decomposition: {mom_date.date()} -> {analysis_date.date()}")
    print("=" * 60)
    print(f"Period 1 Total Bookings: {multi_mom_results.metadata['aggregate_period_1_bookings']:,.0f}")
    print(f"Period 2 Total Bookings: {multi_mom_results.metadata['aggregate_period_2_bookings']:,.0f}")
    print(f"Change:                  {multi_mom_results.metadata['aggregate_delta_bookings']:+,.0f}")
    print(f"\nLenders: {multi_mom_results.metadata['lenders']}")

In [ ]:
# Create and save simple multi-lender MoM aggregate waterfall
if mom_date in available_dates:
    fig, ax = plt.subplots(figsize=(14, 8))
    
    _create_aggregate_waterfall(
        ax=ax,
        summary=multi_mom_results.aggregate_summary,
        period_1_bks=multi_mom_results.metadata['aggregate_period_1_bookings'],
        period_2_bks=multi_mom_results.metadata['aggregate_period_2_bookings'],
        title=f'Multi-Lender MoM Aggregate: {mom_date.date()} -> {analysis_date.date()}'
    )
    
    plt.tight_layout()
    
    # Save PNG
    output_file = output_dir / 'multi_lender_mom_aggregate_simple.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    plt.show()

In [ ]:
# Create and save multi-lender MoM breakdown waterfall (with stacked lender bars)
if mom_date in available_dates:
    fig = create_lender_aggregate_waterfall(
        lender_summaries=multi_mom_results.lender_summaries,
        aggregate_summary=multi_mom_results.aggregate_summary,
        date_a=str(mom_date.date()),
        date_b=str(analysis_date.date()),
        period_1_bks=multi_mom_results.metadata['aggregate_period_1_bookings'],
        period_2_bks=multi_mom_results.metadata['aggregate_period_2_bookings']
    )
    
    # Save PNG
    output_file = output_dir / 'multi_lender_mom_breakdown_waterfall.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    plt.show()

---
## 5. Penetration Decomposition - YoY (Self-Adjusted)

LMDI decomposition of penetration changes using self-adjusted approach:
- **Net Lender Effects** = Gross lender effect - Self-adjustment (denominator impact from lender's own growth)
- **Competitor Effects** = Pure rest-of-market impact (no self-influence)
- **Exact reconciliation** - No residual

Four individual charts saved separately:

1. **Net Lender + Competitor**: 7 net lender effects + 1 total competitor effect
2. **Net All Inclusive**: 7 net effects (net lender + competitor combined per driver)
3. **Net Lender vs Competitor Breakdown**: Stacked waterfall showing both components
4. **Net Volume Inclusive + Split Others**: Net volume (all inclusive) + 6 net lender + remaining competitor

In [ ]:
# Calculate YoY penetration decomposition
if yoy_date in available_dates:
    pen_yoy_results = calculate_penetration_decomposition(
        df=df,
        date_a=str(yoy_date.date()),
        date_b=str(analysis_date.date()),
        lender=LENDER
    )
    
    # Display summary
    print(f"\n{LENDER} YoY Penetration Decomposition")
    print_penetration_decomposition(pen_yoy_results.summary, pen_yoy_results.metadata)

In [ ]:
# YoY Penetration Chart 1: Net Lender + Competitor
if yoy_date in available_dates:
    fig1 = create_penetration_chart_net_lender_plus_competitor(
        summary=pen_yoy_results.summary,
        metadata=pen_yoy_results.metadata,
        lender=LENDER
    )
    
    output_file = output_dir / f'{LENDER.lower()}_yoy_penetration_1_net_lender_plus_competitor.png'
    fig1.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    plt.show()

In [ ]:
# YoY Penetration Chart 2: Net All Inclusive
if yoy_date in available_dates:
    fig2 = create_penetration_chart_net_all_inclusive(
        summary=pen_yoy_results.summary,
        metadata=pen_yoy_results.metadata,
        lender=LENDER
    )
    
    output_file = output_dir / f'{LENDER.lower()}_yoy_penetration_2_net_all_inclusive.png'
    fig2.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    plt.show()

In [ ]:
# YoY Penetration Chart 3: Net Lender vs Competitor Breakdown
if yoy_date in available_dates:
    fig3 = create_penetration_chart_net_lender_vs_competitor(
        summary=pen_yoy_results.summary,
        metadata=pen_yoy_results.metadata,
        lender=LENDER
    )
    
    output_file = output_dir / f'{LENDER.lower()}_yoy_penetration_3_net_lender_vs_competitor.png'
    fig3.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    plt.show()

In [ ]:
# YoY Penetration Chart 4: Net Volume Inclusive + Split Others
if yoy_date in available_dates:
    fig4 = create_penetration_chart_net_volume_inclusive(
        summary=pen_yoy_results.summary,
        metadata=pen_yoy_results.metadata,
        lender=LENDER
    )
    
    output_file = output_dir / f'{LENDER.lower()}_yoy_penetration_4_net_volume_inclusive.png'
    fig4.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    plt.show()

---
## 6. Penetration Decomposition - MoM (Self-Adjusted)

In [ ]:
# Calculate MoM penetration decomposition
if mom_date in available_dates:
    pen_mom_results = calculate_penetration_decomposition(
        df=df,
        date_a=str(mom_date.date()),
        date_b=str(analysis_date.date()),
        lender=LENDER
    )
    
    # Display summary
    print(f"\n{LENDER} MoM Penetration Decomposition")
    print_penetration_decomposition(pen_mom_results.summary, pen_mom_results.metadata)

In [ ]:
# MoM Penetration Chart 1: Net Lender + Competitor
if mom_date in available_dates:
    fig1 = create_penetration_chart_net_lender_plus_competitor(
        summary=pen_mom_results.summary,
        metadata=pen_mom_results.metadata,
        lender=LENDER
    )
    
    output_file = output_dir / f'{LENDER.lower()}_mom_penetration_1_net_lender_plus_competitor.png'
    fig1.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    plt.show()

In [ ]:
# MoM Penetration Chart 2: Net All Inclusive
if mom_date in available_dates:
    fig2 = create_penetration_chart_net_all_inclusive(
        summary=pen_mom_results.summary,
        metadata=pen_mom_results.metadata,
        lender=LENDER
    )
    
    output_file = output_dir / f'{LENDER.lower()}_mom_penetration_2_net_all_inclusive.png'
    fig2.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    plt.show()

In [ ]:
# MoM Penetration Chart 3: Net Lender vs Competitor Breakdown
if mom_date in available_dates:
    fig3 = create_penetration_chart_net_lender_vs_competitor(
        summary=pen_mom_results.summary,
        metadata=pen_mom_results.metadata,
        lender=LENDER
    )
    
    output_file = output_dir / f'{LENDER.lower()}_mom_penetration_3_net_lender_vs_competitor.png'
    fig3.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    plt.show()

In [ ]:
# MoM Penetration Chart 4: Net Volume Inclusive + Split Others
if mom_date in available_dates:
    fig4 = create_penetration_chart_net_volume_inclusive(
        summary=pen_mom_results.summary,
        metadata=pen_mom_results.metadata,
        lender=LENDER
    )
    
    output_file = output_dir / f'{LENDER.lower()}_mom_penetration_4_net_volume_inclusive.png'
    fig4.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    plt.show()

---
## Summary of Saved Files

In [ ]:
# List all saved files
print("\nSaved PNG Files:")
print("=" * 70)
for f in sorted(output_dir.glob('*.png')):
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name:<55} ({size_kb:,.0f} KB)")
print("=" * 70)
print(f"\nTotal files: {len(list(output_dir.glob('*.png')))}")
print(f"Location: {output_dir}")